In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp trait

## Trait
> contains logic for quantitative traits

In [ ]:
#| export

from chewc.core import *
import torch
import attr
from typing import Tuple, Optional, List
from fastcore.test import *
import matplotlib.pyplot as plt
import torch.nn as nn
import pdb

def select_qtl_loci(num_qtl_per_chromosome: int, genome: Genome) -> torch.Tensor:
    """
    Randomly selects loci to be QTLs on each chromosome.

    Args:
    ----
    num_qtl_per_chromosome (int): Number of QTLs to select per chromosome.
    genome (Genome): Genome object containing the chromosome structure.
    device (str): Device to create the tensor on ('cpu' or 'cuda'). Defaults to 'cpu'.

    Returns:
    -------
    torch.Tensor: A boolean tensor indicating which loci are QTLs. 
                  Shape: (number_chromosomes, loci_per_chromosome)
    """
    
    assert num_qtl_per_chromosome <= genome.n_loci_per_chromosome, "too many qtls for this trait given your Genome object"
    assert num_qtl_per_chromosome > 0, "you need at least 1 QTL per chromosome"
    
    qtl_indices = []
    for i in range(genome.n_chromosomes):
        # Randomly sample indices for QTLs on the current chromosome
        chromosome_indices = torch.randperm(genome.n_loci_per_chromosome)[:num_qtl_per_chromosome]
        
        # Create a boolean tensor for the current chromosome, marking QTL positions as True
        chromosome_qtl_flags = torch.zeros(genome.n_loci_per_chromosome, dtype=torch.bool)
        chromosome_qtl_flags[chromosome_indices] = True
        
        qtl_indices.append(chromosome_qtl_flags)
    
    return torch.stack(qtl_indices)

In [ ]:
#|export

class TraitA(nn.Module):
    """
    Represents a trait with only additive genetic effects.

    Args:
        genome (Genome): The genome object.
        target_mean (float): The desired genetic mean for the trait in the founder population.
        target_variance (float): The desired genetic variance for the trait in the founder population.
    """
    
    def __init__(self, genome: Genome, founder_population: Population, target_mean: float, target_variance: float):
        super().__init__()
        self.genome = genome
        self.target_mean = torch.tensor(target_mean, device=self.genome.device)
        self.target_variance = torch.tensor(target_variance, device=self.genome.device)
        self.effects = None  # QTL effects (to be initialized later)
        self.intercept = None
        self.qtl_map =  torch.randint(0, 2, (self.genome.n_chromosomes, self.genome.n_loci_per_chromosome), device=self.genome.device) # random qtl map
        
        # Move computations to the GPU for performance
        with torch.no_grad(): 
            self.sample_initial_effects()
            self.scale_effects(founder_population)

    def _calculate_scaled_additive_dosages(self, genotypes: torch.Tensor) -> torch.Tensor:
        """
        Calculates the scaled additive genotype dosages.

        Args:
            genotypes (torch.Tensor): A tensor representing the genotypes of individuals.
                                     Shape: (n_individuals, ploidy, n_chromosomes, n_loci_per_chromosome).

        Returns:
            torch.Tensor: A tensor of scaled additive dosages. 
                         Shape: (n_individuals, ploidy, n_chromosomes, n_loci_per_chromosome).
        """
        return (genotypes - self.genome.ploidy / 2) * (2 / self.genome.ploidy)

    def sample_initial_effects(self):
        """
        Sample initial values for the genetic effects from a standard normal distribution.
        """
        self.effects = torch.randn((self.genome.n_chromosomes, self.genome.n_loci_per_chromosome), device=self.genome.device) 

    def scale_effects(self, founder_pop: Population):
        """
        Scale the marker effects to achieve the specified genetic variance in the founder population.
        """
        founder_genotypes = founder_pop.get_dosages().float().to(self.genome.device)  # Move to GPU
        scaled_dosages = self._calculate_scaled_additive_dosages(founder_genotypes)
        # Apply QTL map
        scaled_dosages = scaled_dosages * self.qtl_map[None, None, :, :]  
        genetic_values = torch.sum(scaled_dosages * self.effects[None, None, :, :], dim=(2, 3))
        current_mean = genetic_values.mean()
        self.intercept = self.target_mean - current_mean
        initial_variance = torch.var(genetic_values)
        scaling_constant = torch.sqrt(self.target_variance / initial_variance)
        self.effects = self.effects * scaling_constant
        # Intercept is now fixed to the target mean
        
#         current_mean = self.calculate_genetic_values(self.sim_param.founder_pop).mean()
#         self.intercept = self.target_mean - current_mean

    def forward(self, genotypes: torch.Tensor) -> torch.Tensor:
        """
        Calculate the genetic value of individuals

        Args:
            genotypes (torch.Tensor): A tensor representing the genotypes of individuals.
                                     Shape: (n_individuals, ploidy, n_chromosomes, n_loci_per_chromosome).

        Returns:
            torch.Tensor: Genetic values of individuals. Shape: (n_individuals)
        """
        genotypes = genotypes.to(self.genome.device) # Ensure genotypes are on the same device as effects
        scaled_dosages = self._calculate_scaled_additive_dosages(genotypes)
        # Apply QTL map
        scaled_dosages = scaled_dosages * self.qtl_map
        genetic_values = torch.sum(scaled_dosages * self.effects[None, None, :, :], dim=(2, 3)) + self.intercept
        return genetic_values

In [ ]:
#example
#core logic
n_loci_per_chromosome = 10000
n_founders = 500

genome = Genome(n_loci_per_chromosome=n_loci_per_chromosome)
founder_pop = Population()
founder_pop.create_random_founder_population(genome, n_founders=n_founders)


#trait logic
qtl_map = select_qtl_loci(n_loci_per_chromosome, genome)
t = TraitA(genome, founder_pop, 500, 450)


In [ ]:
# #| export 


# def calculate_genetic_variance(founder_pop: torch.Tensor, marker_effects: torch.Tensor, genome: Genome) -> float:
#     """
#     Calculates the additive genetic variance in the founder population.

#     Args:
#         founder_pop (torch.Tensor): Tensor of founder haplotypes. 
#                                     Shape: (n_founders, ploidy, number_chromosomes, loci_per_chromosome)
#         marker_effects (torch.Tensor): Tensor of marker effects. 
#                                        Shape: (number_chromosomes, loci_per_chromosome)
#         genome (Genome): The genome object.

#     Returns:
#         torch.Tensor: The additive genetic variance.
#     """
#     # Convert to float for mean operation to work
#     founder_pop = founder_pop.float()
#     # Calculate allele frequencies in the founder population
#     allele_frequencies = founder_pop.mean(dim=(0, 1))  # Average across founders and ploidy

#     # Calculate the genetic value contributions of each locus
#     locus_contributions = 2 * allele_frequencies * (1 - allele_frequencies) * marker_effects**2

#     # Sum the contributions across all loci to get the total genetic variance
#     genetic_variance = locus_contributions.sum().item()

#     return genetic_variance

In [ ]:
# #| export

# import torch
# import attr
# import numpy as np

# @attr.s(auto_attribs=True)
# class TraitA:
#     """
#     Represents a trait with only additive genetic effects.

#     Attributes:
#     ----------
#     qtl_map (torch.Tensor): A boolean tensor indicating which loci are QTLs. 
#                            Shape: (number_chromosomes, loci_per_chromosome)
#     genome (Genome): The genome object.
#     sim_param (SimParam): Simulation parameters including device.
#     target_mean (float): The desired genetic mean for the trait.
#     target_variance (float): The desired genetic variance for the trait.
#     distribution (str): Distribution type for initial effects. Defaults to 'normal'.
#     intercept (float): The intercept value, calculated during initialization. Defaults to 0.
#     """

#     qtl_map: torch.Tensor 
#     sim_param: SimParam
#     target_mean: float
#     target_variance: float
#     distribution: str = 'normal'
#     intercept: float = 0.0

#     def calculate_scaled_additive_dosages(self, genotypes: torch.Tensor) -> torch.Tensor:
#         """
#         Calculates the scaled additive genotype dosages.

#         Args:
#             genotypes (torch.Tensor): A tensor representing the genotypes of individuals.
#                                      Shape: (n_individuals, ploidy, number_chromosomes, loci_per_chromosome).

#         Returns:
#             torch.Tensor: A tensor of scaled additive dosages. 
#                          Shape: (n_individuals, ploidy, number_chromosomes, loci_per_chromosome).
#         """
#         return (genotypes - self.sim_param.genome.ploidy / 2) * (2 / self.sim_param.genome.ploidy)

#     def sample_initial_effects(self):
#         """
#         Sample initial values for the genetic effects based on the specified distribution.
#         """
#         device = self.sim_param.device
#         if self.distribution == 'normal':
#             # Sample from a standard normal distribution
#             effects = torch.randn(self.sim_param.genome.genetic_map.shape, device=device)
#         elif self.distribution == 'gamma':
#             # Sample from a gamma distribution with specified shape and scale=1
#             effects = torch.distributions.Gamma(self.shape, 1).sample((self.sim_param.genome.genetic_map.shape,)).to(device)
#             # Randomly assign a positive or negative sign
#             signs = torch.tensor(np.random.choice([-1, 1], self.sim_param.genome.genetic_map.shape)).to(device)
#             effects *= signs
#         else:
#             raise ValueError("Unsupported distribution type. Choose 'normal' or 'gamma'.")
#         self.initial_effects = effects
        
#     def scale_genetic_effects(self):
#         """
#         Scale the marker effects to achieve a specified genetic variance in the founder population.

#         Parameters:
#         founder_pop (torch.Tensor): Tensor representing the founder population. Shape: [num_individuals, num_chromosomes, dim1, dim2]
#         marker_effects (torch.Tensor): Tensor of marker effects. Shape: [dim1, dim2]
#         target_variance (float): The target genetic variance to achieve.

#         Returns:
#         torch.Tensor: Scaled marker effects.
#         """
#         device = self.sim_param.device
#         # Step 1: Calculate the genetic value for each individual using the initially sampled marker effects
#         genetic_values = (self.sim_param.founder_pop * self.initial_effects.unsqueeze(0).unsqueeze(0)).sum(dim=[1, 2, 3])
#         # Step 2: Calculate the initial genetic variance
#         initial_variance = genetic_values.var(dim=0)
#         # Step 3: Calculate scaling constant and scale the effects
#         scaling_factor = torch.sqrt(self.target_variance / initial_variance).to(device)
#         scaled_effects = self.initial_effects * scaling_factor
#         self.scaled_effects = scaled_effects


#     def calculate_intercept(self):
#         """
#         Calculate the intercept to achieve the desired mean genetic value of the population.
#         """
#         current_mean = self.calculate_genetic_values(self.sim_param.founder_pop).mean()
#         self.intercept = self.target_mean - current_mean
        
    
    
    
#     def calculate_genetic_values(self, pop):
#         """
#         Calculates the genetic value for each individual in a pop [ind,ploidy,chr,loci]
#         """
#         if self.intercept:
#             return torch.einsum('ijkl,kl->i', self.calculate_scaled_additive_dosages(pop.float()), self.scaled_effects) + self.intercept
#         else:
#             return torch.einsum('ijkl,kl->i', self.calculate_scaled_additive_dosages(pop.float()), self.scaled_effects)


#     def phenotype(self, pop, h2=None, varE=None, reps=1):
#         """
#         Generate phenotypes for the individuals in the population.

#         Args:
#         pop (torch.Tensor): The population tensor.
#         h2 (float, optional): Target narrow-sense heritability. Defaults to None.
#         varE (float, optional): Target environmental variance. Defaults to None.
#         reps (int, optional): Number of repetitions for averaging. Defaults to 1.

#         Returns:
#         torch.Tensor: A tensor of average phenotype values for each individual.
#                       Shape: (n_individuals,)

#         Raises:
#         ValueError: If both h2 and varE are provided or neither are provided.
#         """
#         device = self.sim_param.device
#         pop = pop.to(device)
        
#         if h2 is not None and varE is not None:
#             raise ValueError("Provide either h2 or varE, not both.")
#         if h2 is None and varE is None:
#             raise ValueError("Provide either h2 or varE.")

#         genetic_values = self.calculate_genetic_values(pop)

#         if h2 is not None:
#             # Calculate varE based on target heritability
#             varG = genetic_values.var()
#             varE = (varG / h2) - varG  

#         # Generate phenotypes for each repetition
#         phenotypes = torch.zeros(len(pop), device=device)
#         for _ in range(reps):
#             error = torch.randn(len(pop), device=device) * torch.sqrt(varE)
#             phenotypes += genetic_values + error

#         # Average phenotypes across repetitions
#         phenotypes /= reps

#         return phenotypes


In [ ]:
# #| export

# from typing import List

# def create_multiple_traits(sim_param: SimParam, 
#                            n_traits: int,
#                            target_means: torch.Tensor,
#                            target_variances: torch.Tensor,
#                            correlations: torch.Tensor,
#                            num_qtl_per_chromosome: int,
#                            distribution: str = 'normal') -> List[TraitA]:
#     """
#     Creates multiple additive traits that share the same QTL map.

#     Args:
#         sim_param (SimParam): The SimParam object.
#         n_traits (int): The number of traits to create.
#         target_means (torch.Tensor): A tensor of desired means for each trait. 
#                                       Shape: (n_traits,).
#         target_variances (torch.Tensor): A tensor of desired variances for each trait. 
#                                          Shape: (n_traits,).
#         correlations (torch.Tensor): A 2D correlation matrix for the marker effects between traits. 
#                                      Shape: (n_traits, n_traits).
#         num_qtl_per_chromosome (int): Number of QTLs to select per chromosome (shared by all traits).
#         distribution (str, optional): Distribution to sample initial effects from. 'normal' or 'gamma'. Defaults to 'normal'.

#     Returns:
#         List[TraitA]: A list of TraitA objects, each representing one of the created traits.
#     """
#     device = sim_param.device
    
#     # Check input dimensions
#     n_traits = target_variances.shape[0]
#     test_eq(target_means.shape, (n_traits,))
#     test_eq(target_variances.shape, (n_traits,))
#     test_eq(correlations.shape, (n_traits, n_traits))
    
#     # Sample shared QTL positions
#     qtl_map = select_qtl_loci(num_qtl_per_chromosome, sim_param.genome).to(device)
    
#     # Generate correlated effects 
#     if distribution == 'normal':
#         # Cholesky decomposition for correlated normal variables
#         L = torch.linalg.cholesky(correlations).to(device)
#         uncorrelated_effects = torch.randn((sim_param.genome.number_chromosomes, sim_param.genome.loci_per_chromosome, n_traits), device=device)
#         correlated_effects = torch.matmul(uncorrelated_effects, L.T)
        
#     elif distribution == 'gamma':
#         # Not as straightforward to generate correlated gamma variables directly 
#         # One option is to use copulas (requires additional libraries and complexity)
#         raise NotImplementedError("Correlated gamma distribution for multiple traits is not yet implemented.")

#     else:
#         raise ValueError("Unsupported distribution type. Choose 'normal' or 'gamma'.")

#     # Create TraitA objects
#     traits = []
#     for i in range(n_traits):
#         # Extract effects for the current trait
#         trait_effects = correlated_effects[:, :, i]  
        
#         # Multiply trait effects by the QTL map
#         trait_effects = trait_effects * qtl_map

#         trait = TraitA(
#             qtl_map=qtl_map,
#             sim_param=sim_param,
#             target_mean=target_means[i],
#             target_variance=target_variances[i],
#             distribution=distribution
#         )
#         trait.initial_effects = trait_effects
#         trait.scale_genetic_effects()
#         trait.calculate_intercept()
#         traits.append(trait)

#     return traits


In [ ]:
# ploidy = 2
# number_chromosomes = 10
# loci_per_chromosome = 100
# genetic_map = create_random_genetic_map(number_chromosomes,loci_per_chromosome)
# crop_genome = Genome(ploidy, number_chromosomes, loci_per_chromosome, genetic_map)

# n_founders = 500
# founder_pop = create_random_founder_pop(crop_genome , n_founders)
# simparam = SimParam
# simparam.founder_pop = founder_pop
# simparam.genome = crop_genome


# #add a single additive trait
# qtl_loci = 100
# qtl_map = select_qtl_loci(qtl_loci,simparam.genome)

# ta = TraitA(qtl_map,simparam,5, 5)
# ta.sample_initial_effects()
# ta.scale_genetic_effects()
# ta.calculate_intercept()

# # Create a 2x2 grid of plots
# fig, axs = plt.subplots(2, 2, figsize=(10, 10))  # Adjust figsize as needed

# # Plot 1: Scaled Effects Histogram
# axs[0, 0].hist(ta.scaled_effects.flatten())
# axs[0, 0].set_title('Marker Effects')

# # Plot 2: True Genetic Values Histogram
# genetic_values = ta.calculate_genetic_values(simparam.founder_pop)
# axs[0, 1].hist(genetic_values)
# axs[0, 1].set_title('True Genetic Values')

# # Plot 3: Phenotype Histogram
# phenotypes = ta.phenotype(simparam.founder_pop, h2=0.5)  # Assuming 'h2' is a parameter for heritability in the phenotype method
# axs[1, 0].hist(phenotypes)
# axs[1, 0].set_title('Phenotype')

# # Plot 4: Scatter Plot of Genetic Values vs Phenotypes
# axs[1, 1].scatter(genetic_values, phenotypes)
# axs[1, 1].set_title('Genetic Values vs Phenotype')

# # Display the plots
# plt.tight_layout()  # Adjust layout to prevent overlap
# plt.show()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()